Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline  

import pandas as pd
import numpy as np
import gc

In [2]:
#Reload the output of previous notebook

df = pd.read_pickle('tmp.pkl')
print( df.shape )
df.head(10)

(125780467, 25)


,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_following_count,b_is_verified,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,id,tr
0,A3D61C03DBCE2F920350379414E0048B\t867E9AB76EFF...,0,5,NaN,NaN,2,11,1581472519,0,11005,...,6,False,1568940527,True,0,0,0,0,0,0
1,NaN,1,0,NaN,NaN,0,59,1581124265,1,425,...,185,False,1541013180,True,0,0,0,1581174423,1,0
2,NaN,2,0,NaN,NaN,2,38,1581166895,2,298336,...,72,False,1573996260,False,0,0,0,1581201675,2,0
3,NaN,3,0,NaN,NaN,2,11,1581375781,3,1115,...,440,False,1432084055,True,0,0,0,1581437254,3,0
4,NaN,4,0,NaN,NaN,1,54,1580996019,4,63473,...,32,False,1490688761,False,0,0,0,0,4,0
5,NaN,5,7,NaN,NaN,2,59,1581004518,5,29358,...,928,False,1577292324,True,1581008849,0,0,0,5,0
6,D8788CF06B4D91C8A6CED6296546409D\t49A1E0B17ED6...,6,0,NaN,NaN,1,54,1581209689,6,562,...,928,False,1577292324,True,0,0,0,1581209817,6,0
7,NaN,7,0,NaN,NaN,2,54,1581511341,7,529009,...,952,False,1335110299,False,0,0,0,1581515122,7,0
8,38F9B8B232E7F776F62141A30ED2561A,8,9,NaN,NaN,2,59,1581193484,8,26730,...,522,False,1439811227,False,0,0,0,0,8,0
9,NaN,9,0,NaN,NaN,2,47,1581433245,9,2664,...,140,False,1568691506,False,0,0,0,0,9,0


In [3]:
%%time

df['len_hashtags'] = df['hashtags'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)
df['len_domains']  = df['domains'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)
df['len_links']    = df['links'].apply(lambda x: str(x).count('\t')+1 if not(pd.isnull(x)) else 0)

CPU times: user 4min 58s, sys: 7.34 s, total: 5min 5s
Wall time: 5min 5s


In [4]:
%%time
var = df['domains'].fillna('').values.copy()
gc.collect()

PD = {}
null = ''
PD[null] = [0,0]
count = 1
for vs in var:
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD)

CPU times: user 1min 2s, sys: 1.92 s, total: 1min 4s
Wall time: 1min 4s


381127

In [5]:
list(PD.items())[:10]

[('', [0, 105591537]),
 ('215887F9F98736B07C823045CA9FA2EB', [1, 984]),
 ('3E17824EEB6725B91628B9C13C28894B', [2, 380]),
 ('FECA6F2E8244F2294BD2CE957C0602A9', [3, 834363]),
 ('79F1DCA7DD157C768CCDF4817A0D3999', [4, 170630]),
 ('96A073F6BDEEAEDF58E1C9DD1F41F4F8', [5, 263]),
 ('D0A12E9B144295D771A6A6D058337F65', [6, 388]),
 ('1336DD7055851C3EF96BBF8A6B0447DE', [7, 1235]),
 ('8A67C9075AF148F8C410E56DC9E6BA2C', [8, 10740]),
 ('E91CDEC8DC7ABF30592FA024616FF970', [9, 1227537])]

In [6]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

CPU times: user 5min 1s, sys: 6.26 s, total: 5min 7s
Wall time: 5min 6s


(125780467, [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])

In [7]:
#Get only the most frequent domain
df['domains'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [8]:
%%time
var = df['links'].fillna('').values.copy()
gc.collect()

PD = {}
null = ''
PD[null] = [0,0]
count = 1
for vs in var:
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD),list(PD.items())[:10]

CPU times: user 1min 18s, sys: 2.43 s, total: 1min 20s
Wall time: 1min 20s


(5021417,
 [('', [0, 105591537]),
  ('172DBB18AD23E0AA09074F0C5DBA23D6', [1, 39]),
  ('F4A3AA58F9231C4B9FDE4FCDA1BA557A', [2, 47]),
  ('3A8007779EA9AFDC7196FC405506A61B', [3, 9]),
  ('CBFC53143ED4B60C86C6716CC252687F', [4, 39]),
  ('8D976CF2A1ACFB58A10C4C8E0EE71C9C', [5, 2]),
  ('41DD3CDA748D3F95900DF8E69965DE6C', [6, 1]),
  ('EAED1FB7E20CDE04095A516621E03C24', [7, 1]),
  ('F3D35089AD651FAA1F8FBB020AF10ED7', [8, 8]),
  ('D7BA8D1BE90C19C024DE060DEE6D0E95', [9, 8])])

In [9]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

CPU times: user 5min 15s, sys: 6.12 s, total: 5min 21s
Wall time: 5min 20s


(125780467, [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])

In [10]:
#Get only the most frequent link
df['links'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [11]:
%%time
var = df['hashtags'].fillna('').values.copy()
gc.collect()

PD = {}
null = ''
PD[null] = [0,0]
count = 1
for vs in var:
    if vs != null:
        for v in vs.split('\t'):
            if v not in PD:
                PD[v] = [count,1]
                count +=1
            else:
                x = PD[v]
                x[1] += 1
                PD[v] = x
    else:
        x = PD[null]
        x[1] += 1
        PD[null] = x
        
len(PD),list(PD.items())[:10]

CPU times: user 1min 44s, sys: 2.32 s, total: 1min 46s
Wall time: 1min 46s


(2754023,
 [('', [0, 98303594]),
  ('A3D61C03DBCE2F920350379414E0048B', [1, 1299]),
  ('867E9AB76EFF8C7EF9DE926710DDBC99', [2, 108]),
  ('D8788CF06B4D91C8A6CED6296546409D', [3, 4749]),
  ('49A1E0B17ED69FDB02A736B85F953BFD', [4, 27]),
  ('38F9B8B232E7F776F62141A30ED2561A', [5, 185]),
  ('CD46B9E19F28918E38A893A71D7323ED', [6, 35]),
  ('0C2CE69D7151A003C99145F2071FA76E', [7, 254]),
  ('053074B6C39ADD256B984F5498E431EC', [8, 19494]),
  ('46A5CAAF8FE1D26E72A54A96285C8CAC', [9, 22447])])

In [12]:
%%time
vari = []
for vs in var:
    if vs != null:
        li=[]
        lf=[]
        for v in vs.split('\t'):
            if v!='':
                li.append(PD[v][0])
                lf.append(-PD[v][1])
        vari.append( list(np.array(li)[np.argsort(lf)].astype(np.int32) ) )
    else:
        vari.append( [0] )
del PD
gc.collect()

len(vari), vari[:10]

CPU times: user 7min 29s, sys: 7.87 s, total: 7min 37s
Wall time: 7min 36s


(125780467, [[1, 2], [0], [0], [0], [0], [0], [3, 4], [0], [5], [0]])

In [13]:
#Get only the most frequent hashtag
df['hashtags'] = np.array( [v[0] for v in vari ] ).astype( np.int32 )
gc.collect()
del vari, var
gc.collect()

0

In [14]:
df.tail(10)

,hashtags,tweet_id,media,links,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,...,b_follows_a,reply,retweet,retweet_comment,like,id,tr,len_hashtags,len_domains,len_links
9765311,0,59026843,0,0,0,0,54,1582001186,47578,71562,...,False,0,0,0,1582001700,125780457,2,0,0,0
9765312,0,61226758,0,0,0,1,54,1582091134,394774,17747,...,False,0,0,0,0,125780458,2,0,0,0
9765313,251,50224170,0,0,0,2,54,1581860730,11085502,93812,...,False,0,0,0,0,125780459,2,2,0,0
9765314,0,61226759,9,0,0,1,4,1582086464,129714,4386,...,False,0,0,0,1582091916,125780460,2,0,0,0
9765315,2272312,52324695,7,0,0,2,54,1582095589,46438,17868,...,False,0,0,0,1582102157,125780461,2,1,0,0
9765316,10450,61226760,5,0,0,2,11,1581719195,1007938,1472,...,True,0,0,0,1581726864,125780462,2,6,0,0
9765317,530272,61226761,0,0,0,2,11,1581665518,582771,4236,...,True,0,0,0,1581667126,125780463,2,1,0,0
9765318,0,61226762,0,0,0,2,11,1581602829,7112774,1943,...,True,0,0,0,1581604985,125780464,2,0,0,0
9765319,14428,61226763,0,0,0,2,11,1581722172,188957,4884,...,True,0,0,0,1581726745,125780465,2,4,0,0
9765320,0,61226764,0,0,0,2,11,1581813247,914057,309,...,True,0,0,0,1581813372,125780466,2,0,0,0


In [15]:
df.dtypes

hashtags               int32
tweet_id              uint32
media                  uint8
links                  int32
domains                int32
tweet_type             uint8
language               uint8
timestamp             uint32
a_user_id             uint32
a_follower_count      uint32
a_following_count     uint32
a_is_verified           bool
a_account_creation    uint32
b_user_id             uint32
b_follower_count      uint32
b_following_count     uint32
b_is_verified           bool
b_account_creation    uint32
b_follows_a             bool
reply                 uint32
retweet               uint32
retweet_comment       uint32
like                  uint32
id                    uint32
tr                     int64
len_hashtags           int64
len_domains            int64
len_links              int64
dtype: object

In [16]:
train = df.loc[(df.tr == 0)].copy()
test0 = df.loc[(df.tr == 1)].copy()
test1 = df.loc[(df.tr == 2)].copy()

train.shape, test0.shape, test1.shape

((106254462, 28), (9760684, 28), (9765321, 28))

In [17]:
train.to_parquet( 'train.parquet' )
test0.to_parquet( 'valid.parquet' )
test1.to_parquet( 'test.parquet' )
gc.collect()

0